In [2]:
import os

def get_file_list(dir_name):
    return os.listdir(dir_name)

In [3]:
if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]
    print(file_list)

['news_data\\1_Dae-Ho Lee walk-off homer gives Mariners 4-2 win over Rangers.txt', 'news_data\\1_Korean First Baseman Dae-Ho Lee Becomes Free Agent, Interested In MLB Deal.txt', 'news_data\\1_Lee Dae-ho Announces MLB Aspirations.txt', 'news_data\\1_Lee Dae-ho to Start Spring Training in Arizona.txt', 'news_data\\1_Lee Dae-ho wins MVP in Japan Series.txt', "news_data\\1_Mariners' Lee Dae-ho belts a walk-off homer.txt", 'news_data\\1_Mariners’ Lee Dae-ho gets 1st two-hit game, double.txt', 'news_data\\1_MLB Team Interested In Dae-Ho Lee.txt', "news_data\\1_Seattle Mariners' Newest Signing Dae-Ho Lee Could Become Fan Favorite.txt", 'news_data\\1_SoftBank Hawks Hope to Renew Contract with Lee Dae-ho.txt', 'news_data\\2_Dodgers left with questions after latest Hyun-Jin Ryu setback.txt', 'news_data\\2_Dodgers left-hander Hyun-Jin Ryu expects to be ready for start of season.txt', 'news_data\\2_Dodgers unsure when Hyun-Jin Ryu will throw another bullpen session.txt', 'news_data\\2_Dodgers will